In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 


In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
# include("../../../functions/func_map.jl")
# include("../../../functions/func_stat.jl")
# include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure3/"
mkpath(fig_path)

In [ ]:
analyzer = "chuyu"

In [ ]:
data_info_dict = load("/home/chuyu/Notebooks/project_place_cell/figures/chuyu/figure_data_info.jld2")

data_info_all = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
end

In [ ]:
data_info_all

# Percentage place cell

In [ ]:
function neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
    
    # for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
    region_roi_bool = region_roi_bool[:,findall(region_names .== region_name)][:,1]
    whether_region = falses(n_neuron)
    for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
        if sum(region_roi_bool[neuron_label.==which_neuron]) >0
            whether_region[which_neuron] = true
        end
    end
    return whether_region
end

In [ ]:
which_regions = ["Telencephalon -", "Diencephalon -", "Mesencephalon -", "Rhombencephalon -"]

In [ ]:
@showprogress for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neuron = length(X_all);

    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)



    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)



    place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01));



    whether_region_all = []
    for region_name in which_regions
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
        append!(whether_region_all, [whether_region])
    end

    place_cell_percentage_all = []
    for whether_region in whether_region_all
        region_neurons = findall(whether_region)
        place_cell_percentage = sum(whether_in(region_neurons, place_cell_index_1))./length(region_neurons)
        append!(place_cell_percentage_all, place_cell_percentage)
    end
    
    
    region_place_cell_nr = [length(intersect(place_cell_index_1, findall(whether_region_all[i]))) for i in 1:length(whether_region_all)]
    fraction_place_cell = region_place_cell_nr./length(place_cell_index_1)
    


    
    h5open(joinpath(data_path(ds_save_analyzer_1), "place_cell_percentage.h5"), "w") do file
        file["place_cell_percentage_all"] = Float32.(place_cell_percentage_all)
        file["fraction_place_cell"] = Float32.(fraction_place_cell)
        file["which_regions"] = which_regions

    end;
end

@showprogress for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experiment_filename_2 = data_info[3]
    server_2 = data_info[4]
    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    
    ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
    ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
    ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neuron = length(X_all);

    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)



    all_files = readdir(data_path(ds_save_cy_2))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)



    place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01));



    whether_region_all = []
    for region_name in which_regions
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
        append!(whether_region_all, [whether_region])
    end

    place_cell_percentage_all = []
    for whether_region in whether_region_all
        region_neurons = findall(whether_region)
        place_cell_percentage = sum(whether_in(region_neurons, place_cell_index_1))./length(region_neurons)
        append!(place_cell_percentage_all, place_cell_percentage)
    end
    
    
    region_place_cell_nr = [length(intersect(place_cell_index_1, findall(whether_region_all[i]))) for i in 1:length(whether_region_all)]
    fraction_place_cell = region_place_cell_nr./length(place_cell_index_1)
    


    
    h5open(joinpath(data_path(ds_save_analyzer_2), "place_cell_percentage.h5"), "w") do file
        file["place_cell_percentage_all"] = Float32.(place_cell_percentage_all)
        file["fraction_place_cell"] = Float32.(fraction_place_cell)
        file["which_regions"] = which_regions

    end;
end





In [ ]:
# rerun data
data_info_all = 
[

    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
];

In [ ]:
# for single session exp
@showprogress for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]

    experimenter = data_info[end]

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    


    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neuron = length(X_all);

    # whether individual roi belongs to a certain region
    region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
    region_bool_file = h5open(region_bool_filename, "r")
    global region_names = read(region_bool_file, "region_names")
    global region_roi_bool = read(region_bool_file, "region_roi_bool")
    close(region_bool_file)



    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    n_bins = save_file_name[end-4:end-3]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    place_map_all_1 = HDF5.readmmap(file["place_map_all"])
    specificity_1 = HDF5.readmmap(file["specificity"])
    specificity_population_z_1 = HDF5.readmmap(file["specificity_population_z"])
    specificity_shuffle_z_1 = HDF5.readmmap(file["specificity_shuffle_z"])
    valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
    close(file)



    place_cell_index_1 = intersect(findall(specificity_population_z_1.>3), findall(specificity_shuffle_z_1.>5), findall(specificity_1.>0.01));



    whether_region_all = []
    for region_name in which_regions
        whether_region = neuron_region(region_roi_bool, region_name, neuron_label, n_neuron)
        append!(whether_region_all, [whether_region])
    end

    place_cell_percentage_all = []
    for whether_region in whether_region_all
        region_neurons = findall(whether_region)
        place_cell_percentage = sum(whether_in(region_neurons, place_cell_index_1))./length(region_neurons)
        append!(place_cell_percentage_all, place_cell_percentage)
    end
    
    
    region_place_cell_nr = [length(intersect(place_cell_index_1, findall(whether_region_all[i]))) for i in 1:length(whether_region_all)]
    fraction_place_cell = region_place_cell_nr./length(place_cell_index_1)
    


    
    h5open(joinpath(data_path(ds_save_analyzer_1), "place_cell_percentage.h5"), "w") do file
        file["place_cell_percentage_all"] = Float32.(place_cell_percentage_all)
        file["fraction_place_cell"] = Float32.(fraction_place_cell)
        file["which_regions"] = which_regions

    end;
end

In [ ]:
@pyimport pandas as pd
@pyimport seaborn as sns

@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 

In [ ]:
hot = cm.get_cmap("hot", 100)
alpha_top = 50
cmaplist = [hot(i) for i in 1:(hot.N)]
cmaplist_new = []
length_cmaplist = length(cmaplist)
for (i, color_) in enumerate(cmaplist)
    new_color_ = numpy.copy(color_)
    if i<=alpha_top
        new_color_[4] = (100/alpha_top)*i/length_cmaplist
    end
    append!(cmaplist_new,[new_color_]) 
end
hot_new = mpl_colors.LinearSegmentedColormap.from_list("hot_new",cmaplist_new,100)

In [ ]:
job_hedgehog_rotation_bottom_out = 

[
    ["20230406_100925", 9, "20230406_120147", 9, "chuyu"],
    ["20230429_140408", 8, "20230429_153439", 8, "lorenz"],
]

job_hedgehog_rotation_bottom_out_clean = 

[
    ["20230407_171016", 9, "20230407_184007", 9, "chuyu"],
    ["20230429_194403", 8, "20230429_210211", 8, "jen"],
]


job_hedgehog_landmark_removal = 

[
    ["20230409_141947", 9, "20230409_162256", 9, "drew"],
    ["20230423_103330", 8, "20230423_122640", 8,  "jen"],
    ["20230423_141713", 8, "20230423_155036", 8,  "drew"],
    ["20230423_170228", 8, "20230423_182416", 8,  "drew"],
]

job_hedgehog_circle = 

[

    ["20230422_160839", 8, "20230422_174803", 8, "lorenz"],
    ["20230422_130558", 8, "20230422_143553", 8, "jen"],
    ["20230617_164214", 8, "20230617_183124", 8, "lorenz"],
]

job_landmark_removal = 

[

    ["20230419_143319", 9, "20230419_161007", 9, "chuyu"],
    ["20230420_113755", 9, "20230420_130657", 9, "lorenz"],
    ["20230420_152855", 9, "20230420_165502", 9, "chuyu"],
]

job_boundary_morphing = 

[
    ["20230224_162111", 2, "20230224_171731", 2, "chuyu"],
    ["20230223_135721", 9, "20230223_150240", 9, "chuyu"],
    ["20230618_122231", 8, "20230618_134025", 8, "chuyu"],
    ["20230618_155837", 8, "20230618_170437", 8, "chuyu"],
    
]


datasets_corner_cue = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];



job_corner_cue_change = 

[
    ["20220407_152537", 4, "20220407_170908", 4, "jen"],
    ["20220818_123314", 7, "20220818_145358", 7, "chuyu"],
    ["20220818_163902", 7, "20220818_182702", 7, "chuyu"],
    ["20220819_094333", 1, "20220819_114922", 1, "chuyu"],

]

In [ ]:
data_info_hedgehog =  reduce(vcat, [job_hedgehog_rotation_bottom_out, job_hedgehog_rotation_bottom_out_clean, job_hedgehog_landmark_removal, job_hedgehog_circle])
data_info_corner_cue =  reduce(vcat, [datasets_corner_cue])
data_info_octagon =  reduce(vcat, [job_boundary_morphing])
data_info_square_landmark =  reduce(vcat, [job_landmark_removal])

data_info_circle =  reduce(vcat, [job_corner_cue_change])

In [ ]:
labels = ["Rectangular","Asymmetric", "Square",  "Octagon", "Circle"]

In [ ]:
for (i_data, data_info_all) in enumerate([data_info_corner_cue, data_info_hedgehog,data_info_square_landmark, data_info_octagon, data_info_circle])
    println(length(data_info_all))
end

In [ ]:
fig, ax_all = subplots(1,5,figsize=(5,2))
for (i_data, data_info_all) in enumerate([data_info_corner_cue, data_info_hedgehog,data_info_square_landmark, data_info_octagon, data_info_circle])
    ax = ax_all[i_data]
    place_cell_percentage_all_fish = []
    fraction_tel_place_cell_all_fish = []
for which_data = 1:length(data_info_all)
    data_info = data_info_all[which_data]

    experiment_filename_1 = data_info[1]
    server_1 = data_info[2]
    experimenter = data_info[end]
        
    if labels[i_data] == "Circle"
        experiment_filename_1 = data_info[3]
        server_1 = data_info[4]
        experimenter = data_info[end]
    end

    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    


    # whether individual roi belongs to a certain region
    cell_percentage_filename = joinpath(data_path(ds_save_analyzer_1), "place_cell_percentage.h5")
    cell_percentage_file = h5open(cell_percentage_filename, "r")
    place_cell_percentage_all = read(cell_percentage_file, "place_cell_percentage_all")
    fraction_place_cell = read(cell_percentage_file, "fraction_place_cell")
    which_regions = read(cell_percentage_file, "which_regions")
    close(cell_percentage_file)
        
    append!(place_cell_percentage_all_fish, [place_cell_percentage_all])
    append!(fraction_tel_place_cell_all_fish, [fraction_place_cell])

end
    
    
interested_large_regions = ["tel", "di", "mes", "rhomb"]
region_percentage_all = []
for i in 1:length(place_cell_percentage_all_fish)
    
    region_percentage = Dict()
    for (i_region, region_name) in enumerate(interested_large_regions)
        region_percentage[region_name] = place_cell_percentage_all_fish[i][i_region]
    end
    
    append!(region_percentage_all,[region_percentage])
end

region_fraction_all = []
for i in 1:length(fraction_tel_place_cell_all_fish)
    region_fraction = Dict()
    for (i_region, region_name) in enumerate(interested_large_regions)
        region_fraction[region_name] = fraction_tel_place_cell_all_fish[i][i_region]
    end
    
    append!(region_fraction_all,[region_fraction])
end
    
df = pd.DataFrame(region_percentage_all)
# sns.set_theme(style="whitegrid")
# tips = sns.load_dataset("tips")

sns.stripplot(order = interested_large_regions, data=df,ax=ax, color=hot_new(60), s=3, alpha=0.8)
if i_data == 1
    ax.set_ylabel("# PCs per region / # cells per region")
end
ax.spines["top"].set_visible(false)
ax.spines["right"].set_visible(false)



order = interested_large_regions
for i in 0:length(order)-1
    m = numpy.nanmedian([x[order[i+1]] for x in region_percentage_all])
    ax.plot([i-0.3, i+0.3], [m, m], c="k", lw=2)
end
ax.set_xticks([0,1,2,3])
ax.set_xticklabels(labels= ["Tel.", "Di.", "Mes.", "Rhomb."], rotation=45, ha="right" , rotation_mode="anchor")
# yticks([0,500,1000,1500,2000],[0,"","","",2000])

    
ax.set_title(labels[i_data], fontsize=7)
    
ax.set_ylim(-0.01,0.25)
    
end
tight_layout()


fig.savefig(joinpath(fig_path, "geometry_region_percentage.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)
